In [1]:
! pip install pdfplumber

In [2]:
! pip install pandas

In [10]:
import os
import pdfplumber
import pandas as pd
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage



# Load .env file (ensure OPENAI_API_KEY is present there)
load_dotenv()

from langchain.chat_models import ChatOpenAI
from langchain.agents import create_react_agent
from langgraph.graph import StateGraph, END

In [4]:
llm = ChatOpenAI(temperature=0.2, model="gpt-4o")

C:\Users\kalva\AppData\Local\Temp\ipykernel_17764\4182215775.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.2, model="gpt-4o")


In [5]:
pdf_path = r"D:\AgenticAI\AgenticAI-classes\NIKE, Inc. (NKE) Q4 2025 Earnings Call Transcript.pdf"

with pdfplumber.open(pdf_path) as pdf:
    transcript = "\n".join(
        page.extract_text() for page in pdf.pages if page.extract_text()
    )


In [14]:
extractor_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a financial analyst."),
    ("user", "Extract revenue, YoY growth, margin trends, and guidance from the following earnings call transcript:\n{transcript}")
])

def extractor_agent(state):
    transcript = state["messages"][0]["content"]
    prompt = extractor_prompt.format_prompt(transcript=transcript)
    result = llm.invoke(prompt.to_messages())
    msg = {
        "role": "assistant",
        "content": result.content,
        "name": "extractor"
    }
    state["messages"].append(msg)
    state["next"] = "supervisor"
    return state

sentiment_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analysis assistant for finance."),
    ("user", "Analyze the sentiment (positive, cautious, or negative) of the executive remarks in the transcript:\n{transcript}")
])

def sentiment_agent(state):
    transcript = state["messages"][0]["content"]
    prompt = sentiment_prompt.format_prompt(transcript=transcript)
    result = llm.invoke(prompt.to_messages())
    msg = {
        "role": "assistant",
        "content": result.content,
        "name": "sentiment"
    }
    state["messages"].append(msg)
    state["next"] = "supervisor"
    return state

summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert earnings call summarizer."),
    ("user", "Write a concise, executive-level summary of this earnings call transcript:\n{transcript}")
])

def summary_agent(state):
    transcript = state["messages"][0]["content"]
    prompt = summary_prompt.format_prompt(transcript=transcript)
    result = llm.invoke(prompt.to_messages())
    msg = {
        "role": "assistant",
        "content": result.content,
        "name": "summary"
    }
    state["messages"].append(msg)
    state["next"] = "supervisor"
    return state


In [15]:
def supervisor_node(state):
    agents_called = [m.get("name") for m in state["messages"] if m.get("name")]
    if "extractor" not in agents_called:
        state["next"] = "extractor"
    elif "sentiment" not in agents_called:
        state["next"] = "sentiment"
    elif "summary" not in agents_called:
        state["next"] = "summary"
    else:
        state["next"] = "FINISH"
    return state


In [20]:
workflow = StateGraph(dict)
workflow.add_node("extractor", extractor_agent)
workflow.add_node("sentiment", sentiment_agent)
workflow.add_node("summary", summary_agent)
workflow.add_node("supervisor", supervisor_node)

workflow.set_entry_point("supervisor")

workflow.add_edge("extractor", "supervisor")
workflow.add_edge("sentiment", "supervisor")
workflow.add_edge("summary", "supervisor")

workflow.add_conditional_edges("supervisor", lambda s: s["next"], {
    "extractor": "extractor",
    "sentiment": "sentiment",
    "summary": "summary",
    "FINISH": END   # <-- This must match what your supervisor returns!
})



app = workflow.compile()


In [21]:
initial_state = {
    "messages": [{"role": "user", "content": transcript}],
    "next": "supervisor"
}
final_state = app.invoke(initial_state)


In [22]:
rows = [
    {"Role": m["role"], "Agent": m.get("name", "user"), "Content": m["content"]}
    for m in final_state["messages"]
]
df = pd.DataFrame(rows)
df.style.set_properties(**{"white-space": "pre-wrap"})


In [23]:
summary = {m["name"]: m["content"] for m in final_state["messages"] if m.get("name")}
import json
print(json.dumps(summary, indent=2))


{
  "extractor": "**Revenue:**\n- Q4 2025 revenue was $11.10 billion, which represents a decline of 11.97% year-over-year (YoY).\n\n**Year-over-Year (YoY) Growth:**\n- The revenue for Q4 2025 was down 12% on a reported basis and down 11% on a currency-neutral basis.\n- For the full fiscal year 2025, revenue was down 10% on a reported basis and 9% on a currency-neutral basis.\n\n**Margin Trends:**\n- Gross margins for Q4 2025 declined by 440 basis points to 40.3% on a reported basis. This decline was attributed to higher wholesale discounts, higher discounts in NIKE Factory Stores, supply chain cost deleverage, and channel mix headwinds.\n- SG&A was up 1% on a reported basis, driven by a 15% increase in demand creation, partially offset by a 3% decline in operating overhead.\n\n**Guidance:**\n- For Q1 fiscal 2026, NIKE expects revenues to be down mid-single digits.\n- Gross margins for Q1 fiscal 2026 are expected to be down approximately 350 to 425 basis points, including a 100 basis po

In [26]:
# Extract sentiment and summary messages
sentiment = next(
    (m["content"] for m in final_state["messages"] if m.get("name") == "sentiment"), 
    "Sentiment not found."
)
summary = next(
    (m["content"] for m in final_state["messages"] if m.get("name") == "summary"), 
    "Summary not found."
)

print("==== Sentiment Result ====")
print(sentiment)
print("\n==== Executive Summary ====")
print(summary)


==== Sentiment Result ====
The sentiment of the executive remarks in the NIKE Q4 2025 earnings call transcript can be characterized as cautious. 

1. **Acknowledgment of Challenges**: The executives acknowledge that the financial results are not up to the NIKE standard, with revenue and gross margins declining. They also mention geopolitical volatility, tariff uncertainty, and challenges in specific markets like China.

2. **Focus on Strategic Actions**: There is a strong emphasis on the "Win Now" actions and the sport offense realignment, indicating a strategic pivot to address current challenges. This suggests a cautious optimism about the future, as they believe these actions will lead to improvement.

3. **Mixed Performance**: While there are positive signals, such as the progress in North America and EMEA and the positive response to new product lines, the overall tone reflects caution due to ongoing inventory issues and the impact of tariffs.

4. **Forward-Looking Statements**: T

In [27]:
from IPython.display import display, HTML

mermaid_code = """
flowchart TD
    Start([User provides transcript])
    Supervisor([Supervisor Agent])
    Extractor([Extractor Agent<br/>(Key Metrics)])
    Sentiment([Sentiment Agent])
    Summary([Summary Agent])
    End([Results: Sentiment & Summary])

    Start --> Supervisor
    Supervisor --> Extractor
    Extractor --> Supervisor
    Supervisor --> Sentiment
    Sentiment --> Supervisor
    Supervisor --> Summary
    Summary --> Supervisor
    Supervisor --> End
"""

display(HTML(f"""
<div class="mermaid">
{mermaid_code}
</div>
<script type="module">
import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';
mermaid.initialize({{ startOnLoad: true }});
</script>
"""))
